# mimic LOB - DEMO

Steps before running this notebook : 
1. Launch the server in the '5000' port by running the batch 'LaunchSERVER.cmd' and entering 5000 as port.
2. Visit the server address to make sure the server is running

# 0. Imports

In [1]:
# Imports
from   IPython.display import display, HTML
import numpy as np
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt

# Display options
pd.set_option('display.width', 1000)
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('mode.chained_assignment', None)

In [2]:
import mimicLOB as lob
from mimicLOB.agent.genericAgent import genericAgent

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pprint import pprint

# local server
localserver = "http://127.0.0.1:5000"
server = "http://fdr0903.pythonanywhere.com/"
# Server used
server = server

# 2. Agents
The agents is constantly connected to the distant LOB via the server address.

#### Parameters :
1. distant : if true, the agent must have the server address. If False, he should have the orderbook object passed in the configuration.
2. server : when distant is True, the http address of the server the LOB is running on.
3. id : optional, default is generic. the if is used in the transaction tape.
4. b_record : optional, boolean, default is False. If True, the agent records all his activity : sent orders, executed trades, pending orders ...
5. FIXport : port on which the agent is listening to the LOB messages. Every agent should have a different port.


In [4]:
EURONEXT_config = {'distant' : True,
                   'server'  : server,
                   'id'      : 'market',
                   'FIXport' : 501}

myself_config = {'distant' : True,
                 'server'  : server,
                 'id'      : 'FDR',
                 'b_record': True,
                 'FIXport' : 502}

other_config = {'distant'  : True,
                'server'   : server,
                'id'       : 'Olivier',
                'FIXport' : 503}

euronext = genericAgent(**EURONEXT_config)
myself   = genericAgent(**myself_config)
other    = genericAgent(**otherguy_config)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [4]:
# Only agents with b_record = True
LOB.agentList

NameError: name 'LOB' is not defined

#### The agent with id 'market' can change the LOB configuration and reset it

In [ ]:
euronext.setLOB_tickSize(0.1)

# 3. Get the LOB state
Either agents have access to LOB information

In [7]:
LOBstate = myself.getLOBState()
tickSize =  float(myself.getTickSize())
print(f'tick size : {tickSize}')
if type(LOBstate) == pd.DataFrame:
    if len(LOBstate) >0 :
        LOBstate = LOBstate.set_index('Price').sort_index()
        minPrice = LOBstate.index[0]
        maxPrice = LOBstate.index[-1]
        LOBstate = LOBstate.reindex(np.arange(minPrice, maxPrice+tickSize, tickSize))
        LOBstate.plot.bar(figsize=(20, 7))
    else:
        print('LOB is empty')
else:
    display(LOBstate)

tick size : 0.1
LOB is empty


# 4. Send orders

#### limit sell order @101 & limit buy order @99
Check the LOB state afterwards

In [9]:
# Send Orders
myself.send_sell_limit_order(quantity = 100,
                             price    = 101)

myself.send_buy_limit_order(quantity = 100,
                             price    = 100)
print('Sent orders : ')
display(pd.DataFrame.from_dict(myself.sentorders).T)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

print('\n\nExecuted Trades : ')
display(pd.DataFrame.from_dict(myself.executedtrades).T)

**** I received this limit order **** 
: {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': Decimal('101'), 'trader_id': 'FDR', 'timestamp': 1586619418132199000}

**** I received this limit order **** 
: {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': Decimal('100'), 'trader_id': 'FDR', 'timestamp': 1586619418132199000}

Sent orders : 


,type,side,quantity,price,trader_id,timestamp,order_id
0,limit,ask,100,101,FDR,1586619418132199000,1
1,limit,bid,100,100,FDR,1586619418132199000,2




Pending orders (Orders that are still in the LOB) : 


,type,side,quantity,price,trader_id,timestamp,order_id
1,limit,ask,100,101,FDR,1586619418132199000,1
2,limit,bid,100,100,FDR,1586619418132199000,2




Executed Trades : 


""


#### match orders
Olivier will sell 50 at 100. A transaction will happen.

In [10]:
other.send_sell_limit_order(quantity = 50,
                             price    = 100)

print('Sent orders : ')
display(pd.DataFrame.from_dict(myself.sentorders).T)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

print('\n\nExecuted Trades : ')
display(pd.DataFrame.from_dict(myself.executedtrades).T)

**** I received this limit order **** 
: {'type': 'limit', 'side': 'ask', 'quantity': 50, 'price': Decimal('100'), 'trader_id': 'Olivier', 'timestamp': 1586619418155137500}

TRADE: Time - 1586619418155137500, Price - 100, Quantity - 50, TradeID - FDR, Matching TradeID - Olivier
Sent orders : 


,type,side,quantity,price,trader_id,timestamp,order_id
0,limit,ask,100,101,FDR,1586619418132199000,1
1,limit,bid,50,100,FDR,1586619418132199000,2




Pending orders (Orders that are still in the LOB) : 


,type,side,quantity,price,trader_id,timestamp,order_id
1,limit,ask,100,101,FDR,1586619418132199000,1
2,limit,bid,50,100,FDR,1586619418132199000,2




Executed Trades : 


,party1_id,party1_order_id,party1_side,party2_id,party2_order_id,party2_side,time,traded_price,traded_quantity
0,FDR,2,bid,Olivier,None,ask,1586619418155137500,100,50


# 5. Modify orders

Rules for modification : 
1. If the order is already executed : no new order
2. Any order entered into the Central Order Book may be modified or cancelled prior to its execution. Any increase in the order quantity or change in the limit price shall cause the forfeiture of time priority. (Rule 4202/4 of Euronext)

Rules for cancellation :
1. If the order is already executed : no order cancellation

#### Cancel an order

In [11]:
print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)



Pending orders (Orders that are still in the LOB) : 


,type,side,quantity,price,trader_id,timestamp,order_id
1,limit,ask,100,101,FDR,1586619418132199000,1
2,limit,bid,50,100,FDR,1586619418132199000,2


In [12]:
myself.cancelOrder(side     = 'ask', 
                   order_id = 1)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

**** I received this cancel order **** 
: 1



Pending orders (Orders that are still in the LOB) : 


,order_id,price,quantity,side,timestamp,trader_id,type
2,2,100,50,bid,1586619418132199000,FDR,limit


#### modify an order

In [13]:
myself.modifyOrder(order_id     = 2, 
                   side         = 'bid',
                   new_price    = 5,
                   new_quantity = 56)

print('\n\nPending orders (Orders that are still in the LOB) : ')
display(pd.DataFrame.from_dict(myself.pendingorders).T)

**** I received this modify order **** 
: 2

**** I received this limit order **** 
: {'type': 'limit', 'side': 'bid', 'quantity': 56, 'price': Decimal('5'), 'trader_id': 'FDR', 'order_id': 2, 'timestamp': 1586619421860431800}



Pending orders (Orders that are still in the LOB) : 


,order_id,price,quantity,side,timestamp,trader_id,type
2,2,5,56,bid,1586619421860431800,FDR,limit


# 6. Modify The LOB
One agent type (market) can reset the LOB, dump the tape, and modify the LOB properties 
#### Change tick size

In [ ]:
euronext.setLOB_tickSize(0.1)

#### Remove all pending orders

In [ ]:
euronext.resetLOB_PendingOrders()

#### Remove orders and transactions

In [14]:
euronext.resetLOB()

'DONE'